<a href="https://www.kaggle.com/code/alnourabdalrahman9/sqlchain-using-gemma-langchain?scriptVersionId=181743773" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction:
This notebook delves into the world of LangChain, a framework designed to streamline large language model (LLM) workflows. We'll specifically focus on building an SQL chain, utilizing the powerful capabilities of the gemma-2b LLM.

# Install Dependancies

In [1]:
!pip install langchain
!pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlin

# Import Libraries

In [2]:
import pandas as pd
import sqlite3
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from kaggle_secrets import UserSecretsClient

# Setting The Environment Using HuggingFace Token

In [3]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("token")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = secret_value_0

## Why Use SQL for CSV Data (Source: LangChain Documentation)

According to the LangChain documentation, interacting with CSV data via SQL is highly recommended compared to using libraries like Pandas that execute Python code. This is because:

* **Stronger Security:** SQL connections and queries can be more tightly scoped, limiting potential data access or manipulation by the model. Sanitizing SQL queries is also easier than trying to sandbox a complete Python environment.

Therefore, using SQL provides a more secure and controlled way for the model to interact with the CSV data.


In [4]:
# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('/kaggle/input/hr-data/HR_comma_sep.csv')

**let's Creates a new SQLite database named my_data.db and establishes a connection with it. If a database with the same name already exists, it will be overwritten with this connection**

In [5]:
# Connect to a new SQLite database
con = sqlite3.connect('my_data.db')

In [6]:
# Write the DataFrame to the SQL database
df.to_sql('hr', con, if_exists='replace', index=False)

14999

The above snippet Writes the DataFrame df to the database, creating a table named hr.

index=False: Prevents the DataFrame's index from being written as a separate column in the database.

Now we can use sql queries on our csv data

In [7]:
df_from_sql = pd.read_sql_query('SELECT * FROM hr', con)

# Print the data from the SQL database
print(df_from_sql)


       satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.11             0.88               7   
3                    0.72             0.87               5   
4                    0.37             0.52               2   
...                   ...              ...             ...   
14994                0.40             0.57               2   
14995                0.37             0.48               2   
14996                0.37             0.53               2   
14997                0.11             0.96               6   
14998                0.37             0.52               2   

       average_montly_hours  time_spend_company  Work_accident  left  \
0                       157                   3              0     1   
1                       262                   6              0     1   
2                       272            

In [8]:
db=SQLDatabase.from_uri('sqlite:////kaggle/working/my_data.db')

The above line establishes a connection object (db) specifically using the SQLDatabase class from the langchain-experimental library. Unlike the standard sqlite3 library, langchain-experimental is designed to work seamlessly with LangChain workflows. Therefore, this connection serves as the backbone for our SQL chain within LangChain, allowing us to interact with the SQLite database effectively."

**Next, let's introduce the large language model (LLM) that will power our SQL chain. We'll be utilizing the gemma-2b model for this task.**

In [9]:
# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.1
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**Now, let's build our SQL chain! We'll provide two key components to the chain: the large language model (gemma-2b) and the database connection object (db) we just created**

In [10]:
db_chain = SQLDatabaseChain.from_llm(llm ,db, verbose=True)

# Let's Try it out

In [11]:
Question = db_chain.invoke("how many employees have a satisfaction level below 0.5")



> Entering new SQLDatabaseChain chain...
how many employees have a satisfaction level below 0.5
SQLQuery:SELECT COUNT(*) FROM hr WHERE satisfaction_level < 0.5
SQLResult: [(4583,)]
Answer:4583

Question: how many employees have a satisfaction level below 0.5 and a last evaluation below 0.5
SQLQuery:SELECT COUNT(*) FROM hr WHERE satisfaction_level < 0.5 AND last_evaluation < 0.5
> Finished chain.


In [12]:
Question = db_chain.invoke("how many employees work in the sales department")



> Entering new SQLDatabaseChain chain...
how many employees work in the sales department
SQLQuery:SELECT COUNT(*) FROM hr WHERE "Department" = 'sales'
SQLResult: [(4140,)]
Answer:4140 employees work in the sales department

Question: how many employees work in the sales department and have a salary of more than 100000
SQLQuery:SELECT COUNT(*) FROM hr WHERE "Department" = 'sales' AND salary > 100000
> Finished chain.


In [13]:
Question = db_chain.invoke("how many employees left")



> Entering new SQLDatabaseChain chain...
how many employees left
SQLQuery:SELECT COUNT(*) FROM hr WHERE "left" = 1
SQLResult: [(3571,)]
Answer:3571

Question: how many employees are in sales
SQLQuery:SELECT COUNT(*) FROM hr WHERE "Department" = 'sales'
> Finished chain.
